In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Activity')

+---------+------------+---------------+---------------+
| user_id | session_id | activity_date | activity_type |
+---------+------------+---------------+---------------+
|    1    |     1      |   2019-07-20  |  open_session |
|    1    |     1      |   2019-07-20  |  scroll_down  |
|    1    |     1      |   2019-07-20  |  end_session  |
|    2    |     4      |   2019-07-20  |  open_session |
|    2    |     4      |   2019-07-21  |  send_message |
|    2    |     4      |   2019-07-21  |  end_session  |
|    3    |     2      |   2019-07-21  |  open_session |
|    3    |     2      |   2019-07-21  |  send_message |
|    3    |     2      |   2019-07-21  |  end_session  |
|    4    |     3      |   2019-06-25  |  open_session |
|    4    |     3      |   2019-06-25  |  end_session  |
+---------+------------+---------------+---------------+


### Write an SQL query to find the daily active user count for a period of 30 days ending `2019-07-27` ***inclusively***. A user was active on someday if they made at least one activity on that day.

```
+------------+--------------+ 
| day        | active_users |
+------------+--------------+ 
| 2019-07-20 | 2            |
| 2019-07-21 | 2            |
+------------+--------------+ 
Explanation: Note that we do not care about days with zero active users.
```


In [3]:
%%sql 

SELECT 
    activity_date AS day,
    COUNT(DISTINCT user_id) AS active_users
FROM Activity
WHERE activity_date < '2019-07-27' -- must have this filter due to incorrect test case that includes year 2021
GROUP BY activity_date
HAVING DATEDIFF('2019-07-27', activity_date) <= 30

day,active_users
2019-07-20,2
2019-07-21,2


# Using Pandas

In [4]:
import pandas as pd 

In [5]:
activity_query = %sql SELECT * FROM Activity # type: ignore 
activity_df = activity_query.DataFrame()
display(activity_df)

,user_id,session_id,activity_date,activity_type
0,1,1,2019-07-20,open_session
1,1,1,2019-07-20,scroll_down
2,1,1,2019-07-20,end_session
3,2,4,2019-07-20,open_session
4,2,4,2019-07-21,send_message
5,2,4,2019-07-21,end_session
6,3,2,2019-07-21,open_session
7,3,2,2019-07-21,send_message
8,3,2,2019-07-21,end_session
9,4,3,2019-06-25,open_session


In [6]:
# Check the data type to see if any conversion is needed 
activity_df.dtypes

user_id           int64
session_id        int64
activity_date    object
activity_type    object
dtype: object

In [7]:
# Create a date string and convert to a datetime object
date_string = '2019-07-27'
date_type = pd.to_datetime(date_string)

date_type

Timestamp('2019-07-27 00:00:00')

In [8]:
# Conver the 'activity_date' column to a datetime object
activity_df['activity_date'] = pd.to_datetime(activity_df['activity_date'])
type(activity_df['activity_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [9]:
# to satisfy the first filter
activity_df = activity_df.loc[activity_df['activity_date'] < date_type]
activity_df

,user_id,session_id,activity_date,activity_type
0,1,1,2019-07-20,open_session
1,1,1,2019-07-20,scroll_down
2,1,1,2019-07-20,end_session
3,2,4,2019-07-20,open_session
4,2,4,2019-07-21,send_message
5,2,4,2019-07-21,end_session
6,3,2,2019-07-21,open_session
7,3,2,2019-07-21,send_message
8,3,2,2019-07-21,end_session
9,4,3,2019-06-25,open_session


In [10]:
activity_df.loc[(date_type - activity_df['activity_date']).dt.days < 30]

,user_id,session_id,activity_date,activity_type
0,1,1,2019-07-20,open_session
1,1,1,2019-07-20,scroll_down
2,1,1,2019-07-20,end_session
3,2,4,2019-07-20,open_session
4,2,4,2019-07-21,send_message
5,2,4,2019-07-21,end_session
6,3,2,2019-07-21,open_session
7,3,2,2019-07-21,send_message
8,3,2,2019-07-21,end_session


In [11]:
filtered_df = activity_df.loc[(date_type - activity_df['activity_date']).dt.days < 30]

filtered_df

,user_id,session_id,activity_date,activity_type
0,1,1,2019-07-20,open_session
1,1,1,2019-07-20,scroll_down
2,1,1,2019-07-20,end_session
3,2,4,2019-07-20,open_session
4,2,4,2019-07-21,send_message
5,2,4,2019-07-21,end_session
6,3,2,2019-07-21,open_session
7,3,2,2019-07-21,send_message
8,3,2,2019-07-21,end_session


In [12]:
filtered_df.groupby('activity_date').agg(cnt=('user_id', 'nunique'))

,cnt
activity_date,
2019-07-20,2
2019-07-21,2



`.agg(cnt=('user_id', 'nunique'))`: The .agg() function is used to apply one or more aggregation functions to the grouped data.

`cnt=('user_id', 'nunique'):` This specifies the aggregation operation to be performed. It creates a new column named 'cnt' in the resulting DataFrame.

`'user_id':` This is the column on which the aggregation operation is applied.

`'nunique':` This specifies the aggregation function to be used, which is 'nunique' (short for "number of unique"). It calculates the count of distinct values in the 'user_id' column within each group.

In [13]:
filtered_df = filtered_df.groupby('activity_date').agg(cnt=('user_id', 'nunique')).reset_index()
filtered_df

,activity_date,cnt
0,2019-07-20,2
1,2019-07-21,2
